In [1]:
%run ./0___Reference.ipynb

In [2]:
import numpy as np
import json
import papermill as pm
from concurrent.futures import ThreadPoolExecutor, as_completed

In [3]:
import warnings
import os

# Suppress the warning from the debugger
os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = '1'
warnings.filterwarnings("ignore", message=".*frozen modules.*")

---

# Make core folders

In [4]:
os.makedirs(output_directory, exist_ok=True)

---
---
---

# Run the notebooks

In [5]:
def make_folder(name_folder):
    
    if not os.path.exists(name_folder): os.makedirs(name_folder)

---

In [6]:
def execute_notebook_ALL(notebook_directory, notebook_filename, iteration, parameters, output_directory):
    
    '''
    Define a function to execute a notebook.
    
    Usage: This way, we can implement a multiprocessing pipeline which, when using jupyter notebooks, is much 
        faster than using built functions that paralellize the code. We thus write the code that we wish to run
        multiple parallel instances of into a separate notebook and run it from inside another notebook where it 
        needs to be executed.
    '''

    print(parameters)
    
    # Define the input and output paths for the notebook
    input_notebook_path  = os.path.join(notebook_directory, notebook_filename)
    output_notebook_path = os.path.join(output_directory, f"{notebook_filename}_output_{iteration}.ipynb")
    
    # Execute the notebook with papermill
    pm.execute_notebook(input_notebook_path, output_notebook_path, parameters=parameters)

In [7]:
def Run_the_notebooks(notebook_filenames, notebook_parameters, notebook_directory, output_directory, max_parallel=3):
    
    '''
    Run notebooks in parallel, starting a new one whenever another finishes.
    '''
    
    with ThreadPoolExecutor(max_workers=max_parallel) as executor:
        # Submit each notebook execution as a separate future
        futures = [
            executor.submit(execute_notebook_ALL, notebook_directory, notebook_filename, i, notebook_parameters[i], output_directory)
            for i, notebook_filename in enumerate(notebook_filenames)
        ]
        
        # As each notebook completes, start a new one if available
        for future in as_completed(futures):
            future.result()  # This will raise any exceptions from the notebook execution

---
---
---

In [8]:
running = True

In [ ]:
for ALL_index in range(len(ALL_sizes)):
    
    print(); print(); print(); print("- - - - - - - - - - - - - - - - - - ALL_index", ALL_index); print()
    
    cores_no___1_cube                  = ALL_cores_no___1_cube[                 ALL_index]
    cores_no___2_Gaussian              = ALL_cores_no___2_Gaussian[             ALL_index]
    cores_no___3_Layer                 = ALL_cores_no___3_Layer[                ALL_index]
    cores_no___4_UOD_vals_and_lvls     = ALL_cores_no___4_UOD_vals_and_lvls[    ALL_index]
    cores_no___5_Origins               = ALL_cores_no___5_Origins[              ALL_index]
    cores_no___6_Levels_isolated_pairs = ALL_cores_no___6_Levels_isolated_pairs[ALL_index]
    cores_no___7_Finder                = ALL_cores_no___7_Finder[               ALL_index]

    
    file_path     = ALL_file_paths[    ALL_index]
    size          = ALL_sizes[         ALL_index]
    zPaths        = ALL_zPaths[        ALL_index]
    znos          = ALL_znos[          ALL_index]
    edge_cube_cut = ALL_edge_cube_cuts[ALL_index]
    d_xyz         = ALL_d_xyzs[        ALL_index]
    sigma_ps      = ALL_sigma_ps[      ALL_index]
    sigma_cs      = ALL_sigma_cs[      ALL_index]
    lvls          = ALL_lvls[          ALL_index]
    Density_uods  = ALL_Density_uods[  ALL_index]
    run_Origins   = ALL_run_Origins[   ALL_index]
    run_MKs       = ALL_run_MKs[       ALL_index]
    save1percs    = ALL_save1percs[    ALL_index]
    
    

    
    if True:
        print("- - - - - - - - - - - - - - - - - - Cube")
        
        notebook_filenames = []; notebook_parameters = []
        for i0 in range(len(zPaths)):
            zPath        = zPaths[i0]
            zno          = znos[i0]
            file_path_zP = file_path+"z_"+zPath+"/"; make_folder(file_path_zP)
            notebook_filenames.append("1___Cube.ipynb")
            notebook_parameters.append({"basePath":      basePath,
                                        "size":          size,
                                        "zno":           zno,
                                        "ecc":           np.array2string(np.array(edge_cube_cut)*1000, separator=","),
                                        "d_xyz":         d_xyz,
                                        "file_path_zP":  file_path_zP,
                                        "m_DM":          m_DM})
        
        if running:
            Run_the_notebooks(notebook_filenames, notebook_parameters, notebook_directory, output_directory, cores_no___1_cube)
        else:
            for nbp in notebook_parameters: print(nbp)



    
    if True:
        print("- - - - - - - - - - - - - - - - - - Gaussian")
        
        notebook_filenames = []; notebook_parameters = []
        for z in range(len(sigma_ps)):
            zPath             = zPaths[z]
            file_path_zP      = file_path+"z_"+zPath+"/"
            for s in range(len(sigma_ps[z])):
                sigma_p       = sigma_ps[z][s]
                sigma_c       = sigma_cs[z][s]
                file_path_Sgm = file_path_zP+"Sgm___"+str(sigma_p)+"/"; make_folder(file_path_Sgm)
                notebook_filenames.append("2___Gaussian.ipynb")
                notebook_parameters.append({"sigma_c":       sigma_c,
                                            "file_path_zP":  file_path_zP,
                                            "file_path_Sgm": file_path_Sgm})
            
        if running and (len(notebook_filenames) != 0):
            Run_the_notebooks(notebook_filenames, notebook_parameters, notebook_directory, output_directory, cores_no___2_Gaussian)
        else:
            for nbp in notebook_parameters: print(nbp)



    
    if True:
        print("- - - - - - - - - - - - - - - - - - Layer")
        
        notebook_filenames = []; notebook_parameters = []
        for z in range(len(lvls)):
            zPath                 = zPaths[z]
            file_path_zP          = file_path+"z_"+zPath+"/"
            for s in range(len(lvls[z])):
                sigma_p           = sigma_ps[z][s]
                file_path_Sgm     = file_path_zP+"Sgm___"+str(sigma_p)+"/"
                for l in range(len(lvls[z][s])):
                    lvl           = lvls[z][s][l]
                    file_path_Lvl = file_path_Sgm+"Lvl___"+str(lvl)+"/"; make_folder(file_path_Lvl)
                    notebook_filenames.append("3___Layer.ipynb")
                    notebook_parameters.append({"lvl":           lvl,
                                                "file_path_Sgm": file_path_Sgm,
                                                "file_path_Lvl": file_path_Lvl})
        
        if running and (len(notebook_filenames) != 0):
            Run_the_notebooks(notebook_filenames, notebook_parameters, notebook_directory, output_directory, cores_no___3_Layer)
        else:
            for nbp in notebook_parameters: print(nbp)




    if True:
        print("- - - - - - - - - - - - - - - - - - UOD_vals_and_lvls")

        notebook_filenames = []; notebook_parameters = []
        for z in range(len(lvls)):
            zPath                   = zPaths[z]
            file_path_zP            = file_path+"z_"+zPath+"/"
            for s in range(len(lvls[z])):
                sigma_p             = sigma_ps[z][s]
                file_path_Sgm       = file_path_zP+"Sgm___"+str(sigma_p)+"/"
                for l in range(len(lvls[z][s])):
                    lvl             = lvls[z][s][l]
                    Density_uod_l   = Density_uods[z][s][l]
                    file_path_Lvl   = file_path_Sgm+"Lvl___"+str(lvl)+"/"
                    if len(Density_uod_l) != 0:
                        notebook_filenames.append("4___UOD_vals_and_lvls.ipynb")
                        notebook_parameters.append({"lvl":           lvl,
                                                    "Density_uod_l": np.array2string(np.array(Density_uod_l), separator=","),
                                                    "file_path_Sgm": file_path_Sgm,
                                                    "file_path_Lvl": file_path_Lvl})
            
        if running and (len(notebook_filenames) != 0):
            Run_the_notebooks(notebook_filenames, notebook_parameters, notebook_directory, output_directory, cores_no___4_UOD_vals_and_lvls)
        else:
            for nbp in notebook_parameters: print(nbp)
        


    
    if True:
        print("- - - - - - - - - - - - - - - - - - Origins")

        notebook_filenames = []; notebook_parameters = []
        for z in range(len(lvls)):
            zPath                   = zPaths[z]
            file_path_zP            = file_path+"z_"+zPath+"/"
            for s in range(len(lvls[z])):
                sigma_p             = sigma_ps[z][s]
                file_path_Sgm       = file_path_zP+"Sgm___"+str(sigma_p)+"/"
                for l in range(len(lvls[z][s])):
                    lvl             = lvls[z][s][l]
                    file_path_Lvl   = file_path_Sgm+"Lvl___"+str(lvl)+"/"
                    if run_Origins[z][s][l]:
                        notebook_filenames.append("5___Origins.ipynb")
                        notebook_parameters.append({"lvl":           lvl,
                                                    "file_path_Sgm": file_path_Sgm,
                                                    "file_path_Lvl": file_path_Lvl})
            
        if running and (len(notebook_filenames) != 0):
            Run_the_notebooks(notebook_filenames, notebook_parameters, notebook_directory, output_directory, cores_no___5_Origins)
        else:
            for nbp in notebook_parameters: print(nbp)



    
    if True:
        print("- - - - - - - - - - - - - - - - - - Levels isolated pairs")

        notebook_filenames = []; notebook_parameters = []
        for z in range(len(lvls)):
            zPath                   = zPaths[z]
            file_path_zP            = file_path+"z_"+zPath+"/"
            for s in range(len(lvls[z])):
                sigma_p             = sigma_ps[z][s]
                file_path_Sgm       = file_path_zP+"Sgm___"+str(sigma_p)+"/"
                for l in range(len(lvls[z][s])):
                    lvl             = lvls[z][s][l]
                    file_path_Lvl   = file_path_Sgm+"Lvl___"+str(lvl)+"/"
                    if len(run_MKs[z][s][l]) != 0:
                        make_folder(file_path_Lvl+"dict_cells_pairs/")
                        make_folder(file_path_Lvl+"dict_cells_isolated/")
                        notebook_filenames.append("6___Levels_isolated_pairs.ipynb")
                        notebook_parameters.append({"lvl":           lvl,
                                                    "file_path_Lvl": file_path_Lvl})
        
        if running and (len(notebook_filenames) != 0):
            Run_the_notebooks(notebook_filenames, notebook_parameters, notebook_directory, output_directory, cores_no___6_Levels_isolated_pairs)
        else:
            for nbp in notebook_parameters: print(nbp)



        
    if True:
        print("- - - - - - - - - - - - - - - - - - Finder")

        notebook_filenames = []; notebook_parameters = []
        for z in range(len(lvls)):
            zPath                     = zPaths[z]
            file_path_zP              = file_path+"z_"+zPath+"/"
            for s in range(len(lvls[z])):
                sigma_p               = sigma_ps[z][s]
                file_path_Sgm         = file_path_zP+"Sgm___"+str(sigma_p)+"/"
                for l in range(len(lvls[z][s])):
                    lvl               = lvls[z][s][l]
                    file_path_Lvl     = file_path_Sgm+"Lvl___"+str(lvl)+"/"
                    for d in range(len(Density_uods[z][s][l])):
                        [Density_ud, Density_od] = Density_uods[z][s][l][d]
                        for m in range(len(run_MKs[z][s][l][d])):
                            MK        = run_MKs[   z][s][l][d][m]
                            save1perc = save1percs[z][s][l][d][m]
                            file_path_Duod = file_path_Lvl+"D___["+str(Density_ud)+"_"+str(Density_od)+"]/"
                            file_path_MK = file_path_Duod+MK+"/"
                            make_folder(file_path_MK)
                            notebook_filenames.append("7___Finder.ipynb")
                            notebook_parameters.append({"MK":                 MK,
                                                        "save_perc":          save1perc,
                                                        "Density_ud":         Density_ud,
                                                        "Density_od":         Density_od,
                                                        "times_we_tried_max": times_we_tried_max,
                                                        "file_path_Lvl":      file_path_Lvl,
                                                        "file_path_MK":       file_path_MK})


        
        if running and (len(notebook_filenames) != 0):
            Run_the_notebooks(notebook_filenames, notebook_parameters, notebook_directory, output_directory, cores_no___7_Finder)
        else:
            for nbp in notebook_parameters: print(nbp)




- - - - - - - - - - - - - - - - - - ALL_index 0

- - - - - - - - - - - - - - - - - - Cube
{'basePath': '..', 'size': 128, 'zno': '135', 'ecc': '[[    0,75000],\n [    0,75000],\n [    0,75000]]', 'd_xyz': 585.9375, 'file_path_zP': '../Modified_Data_128/z_00/', 'm_DM': 0.0338857141965349}


Executing:   0%|          | 0/13 [00:00<?, ?cell/s]

---
---
---